### 파이썬 트리 클래스 작성
abstract class로 작성. 다수의 not implemented 메소드가 포함되어있으므로 하위 클래스에서 반드시 구현 해야한다.  
iter()를 preorder 기반의 iterator return을 하도록 한다. preorder, postorder, 큐를 이용한 level order 구현  
이진 트리가 아닌 일반 트리의 경우 자식 노드가 여러개가 될 수 있고 이때 preorder, postorder은 가능하지만 inoder은 순서를 정하기 힘들다.  
이진트리에서 inoder의 경우 왼쪽자식->자신->오른쪽자식 순이지만 자식이 여러개가 되면 이 순서를 정할 수 없다.

In [1]:
from exceptions import Empty
import collections

class LinkedQueue: #level order를 위한 연결리스트 큐
    #노드 클래스
    class _Node:
        __slots__ = '_element', '_next'

        def __init__(self, element, next):
            self._element = element
            self._next = next
    ############################################################

    def __init__(self):
        self._head = None
        self._tail = None
        self._size = 0

    def __len__(self):
        return self._size

    def is_empty(self):
        return self._size == 0

    def first(self):
        if self.is_empty():
            raise Empty('Queue is empty')
        return self._head._element

    def dequeue(self):
        if self.is_empty():
            raise Empty('Queue is empty')
        answer = self._head._element
        self._head = self._head._next
        self._size -= 1
        if self.is_empty():
            self._tail = None #큐가 비게되면 tail을 None으로
        return answer

    def enqueue(self, e):
        newest = self._Node(e, None)
        if self.is_empty():
            self._head = newest
        else:
            self._tail._next = newest
        self._tail = newest
        self._size += 1

    def __str__(self):
        A = []
        ptr = self._head
        while ptr is not None:
            A.append(ptr._element)
            ptr = ptr._next
        return f"{str(A)}, front = {A[0]}, rear = {A[-1]}"

In [3]:
class Tree: #not implemented 메소드들은 하위 클래스에서 작성해야한다.
    class Position:
        def element(self): #특정 엘리멘트를 넘겨주는 메소드
            raise NotImplementedError('must be implemented by subclass')
        
        def __eq__(self, other): #Position이 같은지 확인하는 메소드
            raise NotImplementedError('must be implemented by subclass')
            
        def __ne__(self, other): #self와 other이 다르면 not리턴하는 메소드
            return not (self == other)
        
    def root(self): #루트를 나타내는 메소드
        raise NotImplementedError('must be implemented by subclass')
        
    def parent(self, p): #부모를 나타내는 메소드
        raise NotImplementedError('must be implemented by subclass')
    
    def num_children(self, p): #자식의 수를 나타내는 메소드
        raise NotImplementedError('must be implemented by subclass')
        
    def children(self, p): #자식을 나타내는 메소드
        raise NotImplementedError('must be implemented by subclass')
        
    def __len__(self):
        raise NotImplementedError('must be implemented by subclass')
        
    #여기까지가 not implemented 메소드들
        
    def is_root(self, p): #p가 루트인지 확인하는 메소드
        return self.root() == p
    
    def is_leaf(self, p): #p가 리프노드인지 확인하는 메소드
        return self.num_children(p) == 0
    
    def is_empty(self):
        return len(self) == 0
    
    def depth(self, p): #루트에서 얼마나 떨어져있는지 확인하는 메소드. 레벨과 비슷하나 루트는 0으로 하여 레벨-1과 같다
        if self.is_root(p):
            return 0
        else:
            return 1+self.depth(self.parent(p))
        
    def _height1(self): #전체 트리의 높이를 구하는 메소드
        #모든 리프노드의 depth를 구하고 그 중 가장 큰 것 +1 리턴. 시간이 오래걸린다.
        return max(self.depth(p) for p in self. positions() if self.is_leaf(p))
    
    def _height2(self, p): #p의 높이를 구하는 메소드
        #자식 노드의 height2중 가장 큰 것 +1 리턴
        if self.is_leaf(p):
            return 0
        else:
            return 1+max(self._height2(c) for c in self.children(p))
    
    def height(self, p = None): #p가 None일경우 전체 트리의 높이를 구하는 메소드
        if p is None:
            p = self. root()
        return self._height2(p)
    
    def __iter__(self): #iter메소드. 아래의 positions메소드를 통해 preorder로 다음 노드의 정보를 준다.
        for p in self.positions():
            yield p.element()
            
    def positions(self): #preorder로 다음 노드를 리턴하는 메소드
        return self.preorder()
    
    def preorder(self): #안비었으면 preorder주는 메소드
        if not self.is_empty():
            for p in self._subtree_preorder(self.root()):
                yield p
                
    def _subtree_preorder(self, p): #실질적인 preorder메소드
        yield p #자신을 먼저 주고
        for c in self.children(p): #자신의 자식 노드들의 아래 트리를 다시 preorder하면서 값을 준다.
            for othre in self._subtree_preorder(c):
                yield other
                
    def postorder(self): #postorder메소드
         if not self.is_empty():
            for p in self._subtree_postorder(self.root()):
                yield p
                
    def _subtree_postorder(self, p): #실질적인 postorder메소드
        for c in self.children(p): #자신의 자식 노드들의 아래 트리를 다시 preorder하면서 값을 준다.
            for othre in self._subtree_postorder(c):
                yield other
        yield p #자신을 가장 마지막
        
    def breadthfirst(self): #트리를 루트부터 레벨순으로 큐에 추가하며 하나씩 꺼내는 메소드
        if not self.is_empty():
            fringe = LinkedQueue()
            fringe.enqueue(self.root())#루트를 큐에 넣고
            while not fringe.is_empty():
                p = fringe.dequeue() #큐에서 하나꺼내 반환
                yield p
                for c in self.children(p): #꺼내는 노드의 children을 큐에 모두 추가.
                    fringe.enqueue(c)
            #큐가 빌때 까지 반복.

### BinaryTree
Tree클래스를 상속하며 마찬가지로 abstract class로 작성. 다수의 not implemented 메소드가 포함되어있으므로 하위 클래스에서 반드시 구현 해야한다.

In [5]:
class BinaryTree(Tree): #Tree클래스 상속 받는다
    def left(self, p): #왼쪽 자식 메소드
        raise NotImplementedError('must be implemented by subclass')
        
    def right(self, p): #오른쪽 자식 메소드
        raise NotImplementedError('must be implemented by subclass')
        
    def sibling(self, p): #형제 노드 구하는 메소드
        parent = self.parent(p)
        if parent is None: #부모가 없으면 루트노드->형제 없음
            return None
        else:
            if p == self.left(parent):
                return self.right(parent)
            else:
                return self.left(parent)
            
    def children(self, p):
        if self.left(p) is not None:
            yield self.left(p)
        if self.right(p) is not None:
            yield self.right(p)
            
    def inorder(self): #이진트리는 inorder가능
        if not self.is_empty():
            for p in self._subtree_inorder(self.root()):
                yield p
    def _subtree_inorder(self, p):
        if not self.left(p) is not None:
            for other in self._subtree_inorder(self.left(p)):
                yield other
        yield p
        if self.right(p) is not None:
            for other in self._subtree_inorder(self.right(p)):
                yield other
                
    def positions(self): #그냥 Tree클래스에서 preorder였던걸 이진 트리에서는 inorder로 변경(오버라이드)
        return self.inorder() #이진 검색 트리에서 inorder로 정렬된 순서로 출력 가능하기 때문

노드는 element, parent, left, right를 가진다.  
Position은 container, node의 구조. Tree의 instance변수는 root와 size.  
add_root(e), add_left(p,e), add_right(p,e), replace(p,e), delete(p), attach(p, T1, T2)의 메소드를 추가

In [7]:
class LinkedBinaryTree(BinaryTree): #연결리스트로 이진트리
    class _Node:
        __slots__='_element', '_parent', '_left', '_right'
        def __init__(self, element, parent = None, left = None, right = None):
            self._element = element
            self._parent = parent
            self._left = left
            self._right = right
            
    class Position(BinaryTree.Position):
        def __init__(self, container, node):
            self._container = container
            self._node = node
            
        def element(self):
            return self._node._element
        
        def __eq__(self, other):
            return type(other) is type(self) and other._node is self._node
        
    def _validate(self, p): #p가 트리에 쓰는 노드라면 노드를 반환하는 메소드
        if not isinstance(p, self.Position):
            raise TypeError('p must be proper Position type')
        if p._container is not self:
            raise ValueError('p does not belong to this container')
        if p._node._parent is p._node: #트리에서 뺀 노드는 부모를 자기자신을 가르키도록 한다.
            raise ValueError('p is no longer valid')
        return p._node
    
    def _make_position(self, node): #노드의 포지션을 만들어 반환한다.
        return self.Position(self, node) if node is not None else None
        
    def __init__(self):
        self._root = None
        self._size = 0
        
    def __len__(self):
        return self._size
    
    def root(self):
        return self._make_position(self._root) #루트의 포지션을 만들어 넘겨줌
    
    def parent(self, p):
        node = self._validate(p)
        return self._make_position(node._parent)
    
    def left(self, p):
        node = self._validate(p)
        return self._make_position(node._left)
    
    def right(self, p):
        node = self._validate(p)
        return self._make_position(node._right)
    
    def num_children(self, p):
        node = self._validate(p)
        count = 0
        if node._left is not None:
            count += 1
        if node._right is not None:
            count += 1
        return count
    
    def _add_root(self, e):
        if self._root is not None:
            raise ValueError('Root exists')
        self._size = 1
        self._root = self._Node(e)
        return self._make_position(self._root)
    
    def _add_left(self, p, e):
        node = self._validate(p)
        if node._left is not None:
            raise ValueError('Left child exists')
        self._size += 1
        node._left = self._Node(e, node)
        return self._make_position(node._left)
    
    def _add_right(self, p, e):
        node = self._validate(p)
        if node._right is not None:
            raise ValueError('Right child exists')
        self._size += 1
        node._right = self._Node(e, node)
        return self._make_position(node._right)
    
    def _replace(self, p, e):
        node = self._validate(p)
        old = node._element
        node._element = e
        return old
    
    def _delete(self, p):
        node = self._validate(p)
        if self.num_children(p) == 2:
            raise ValueError('Position has two children')
        child = node._left if node._left else node._right
        if child is not None:
            child._parent = node._parent
        if node is self._root:
            self._root = child
        else:
            parent = node._parent
            if node is parent._left:
                parent._left = child
            else:
                parent._right = child
        self._size -= 1
        node._parent = node
        return node._element
    
    def _attach(self, p, t1, t2):
        node = self._validate(p)
        if not self.is_leaf(p):
            raise ValueError('position must be leaf')
        if not type(self) is type(t1) is type(t2):
            raise TypeError('Tree types must match')
        self._size += len(t1) + len(t2)
        if not t1.is_empty():
            t1._root._parent = node
            node._left = t1._root
            t1._root = None
            t1._size = 0
        if not t2.is_empty():
            t2._root._parent = node
            node._right = t2._root
            t2._root = None
            t2._size = 0